# <font color='yellowgreen'>Merged model inference + FTS-OBP evaluation</font>

In [1]:
import gc
from pynvml import *
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from util.prompt import all_prompt_dict
from util.absa_evaluator import ABSAEvaluator
from util.dictToExcel import *

from datetime import date
today = date.today().strftime("%Y-%m-%d") 

#===================================================================================
# User input area
#-----------------------------------------------------------------------------------

##########################################
# True = Use 4 examples/task (total = 20);   False = use the 300 example/task full test dataset (total = 1500)
try_small_sample_with_this_script = True  
##########################################

use_phi4 = True  # if False, use Qwen2.5-1.5B-Instruct; otherwise use Phi-4-mini-instruct

# model_checkpoint_path = '../downloaded_from_HF/EduRABSA_SLM_v1_SLERP_phi4mini'
model_checkpoint_path = '../model/LORA_SFT/merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot'

task_list = ['OE', 'AOPE', 'AOC', 'ASTE', 'ASQE']
prompt_version = 'V1_fewshot' # 'V1_zeroshot'

model_str = 'PHI4' if use_phi4 else 'QW2.5'

is_multitask = len(task_list) > 1

# Set up evaluator and evaluation output dict
prompt_dict = all_prompt_dict[prompt_version]
evaluator = ABSAEvaluator(prompt_dict, equal_weights=True, partial_category_score=0.3, allow_partial_category_for_unit_match=False)

#-----------------------------------------------------------------------------------
handle_format_error = False  # If False, incorrect output format, e.g. additional characters around array '[', ']' will not be handled. 

dataset_subdir = '4_shot_prompt' if prompt_version == 'V1_fewshot' else '0_shot_prompt'
local_DEV_TEST_hf_dataset_dir = f"../datasets/EduRABSA/EduRABSA_SLM_v1_Test_data/{dataset_subdir}"

#============================================================================================================================================
# More stable parameters below:
#============================================================================================================================================

evaluation_model_to_gpu = True
evaluation_device_map = {"": 0} if evaluation_model_to_gpu else 'cpu'
torch_dtype = torch.bfloat16
use_deepspeed = False

#----------------------------------------------------------------------------------
# End of main user input area (pls also adjust configs below if needed)
#===================================================================================
# Auto-produced strings, no need to edit
#. . . . . . . . . . . . . . . . . . . . . . . .
# accelerator_config = "config/accelerate_config_DS.json" if use_deepspeed else "config/accelerate_config_noDS.json"

finetuned_model_id = model_checkpoint_path.replace('../uploaded_to_HF/','').replace('../model/LORA_SFT/','').split(' ')[0].strip()

test_input_dict_dir = f"../OUTPUTS/OUTPUT_1_final_test_output_files/_demo/test_input_{finetuned_model_id}_({today}).py"
test_xlsx_file_dir = f"../OUTPUTS/OUTPUT_2a_final_test_output_FTS-0BP_evaluation/_demo/test_results_{finetuned_model_id}_({today}).xlsx"

#===============================================
# More settings for manual editing

evaluation_setting = {
    "eval_setting_view_sample": True,
    "eval_setting_view_size": 50,
    "handle_format_error": handle_format_error,
    "eval_setting_show_table": False,
}

# inference config for evaluation
generation_args = {"max_new_tokens": 1024,
                   "return_full_text": False,
                   "temperature": None, # or a float after setting 'do_sample=True' 
                   "top_p": None,
                   "top_k": None,
                   "do_sample": False
                   }

evaluation_model_kwargs = {
    'use_cache': False,
    'trust_remote_code': False,  # See issue here: https://github.com/huggingface/transformers/issues/32365
    'attn_implementation': 'flash_attention_2',
    'torch_dtype': torch_dtype,  # loaded pre-trained model torch dtype
    'device_map': evaluation_device_map,  # Must be None if using DeepSpeed Zero3
}

#===================================================================================
sheetname_df_dict = dict()

#===============================================
# Set up accelerator for training
# os.environ["ACCELERATE_CONFIG_FILE"] = accelerator_config
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

accelerator = Accelerator(cpu=not evaluation_model_to_gpu)
device = accelerator.device
print(f"\033[32m\n{'=' * 20}\n Device =  {device} \n{'=' * 20}\033[0m\n")

#===============================================
# Check GPU memory (baseline usage)

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"\033[35mGPU memory occupied: {info.used//1024**2} MB\033[0m\n")

print_gpu_utilization()
# Clear memory
gc.collect()
torch.cuda.empty_cache()  # This is key for releasing GPU memory
torch.cuda.synchronize()  # Ensure all CUDA operations are complete
# Check GPU memory (baseline usage)
print_gpu_utilization()

#===============================================
#===============================================
green_bg = '\033[1;30;42m'
red_bg = '\033[1;30;41m'
yellow_bg = '\033[1;30;43m'
blue_bg = '\033[1;30;44m'
pink_bg = '\033[1;30;45m'

print(f"\nUsing model:           {green_bg if use_phi4 else yellow_bg}Merged_LoRA_{model_str}\033[0m\n\n"
      
      f"is_multitask:          {blue_bg if is_multitask else pink_bg}{is_multitask}\033[0m\n\n"
      
      f"merged model dir:       \033[33m{model_checkpoint_path}\033[0m\n\n"
      
      f"evaluate on GPU:       \033[36m{evaluation_model_to_gpu}\033[0m\n\n"

      f"Local TEST dataset     \033[37m{local_DEV_TEST_hf_dataset_dir}\033[0m\n\n"
      
      f"TEST input dict dir:   \033[34m{test_input_dict_dir}\033[0m\n"
      f"TEST output dir:       \033[32m{test_xlsx_file_dir}\033[0m\n\n"
      )


 Device =  cuda 

GPU memory occupied: 1086 MB

GPU memory occupied: 1342 MB


Using model:           Merged_LoRA_PHI4

is_multitask:          True

merged model dir:       ../model/LORA_SFT/merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot

evaluate on GPU:       True

Local TEST dataset     ../datasets/EduRABSA/EduRABSA_SLM_v1_Test_data/4_shot_prompt

TEST input dict dir:   ../OUTPUTS/OUTPUT_1_final_test_output_files/_demo/test_input_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).py
TEST output dir:       ../OUTPUTS/OUTPUT_2a_final_test_output_FTS-0BP_evaluation/_demo/test_results_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).xlsx





# <font color='cornflowerblue'>Load tokenizer & add custom tokens</font>

In [2]:
################################
# Load tokenizer 
################################
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_path)
tokenizer.model_max_length = generation_args['max_new_tokens']
# tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
# tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

pre_len = len(tokenizer)

################################
# Check existing special tokens
################################
print(tokenizer, f"\n{'=' * 80}\n")

GPT2TokenizerFast(name_or_path='../model/LORA_SFT/merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot', vocab_size=200019, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	199999: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200018: AddedToken("<|endofprompt|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200019: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200020: AddedToken("<|end|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200021: AddedToken("<|user|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=Tr


# <font color='cornflowerblue'>Data processing</font>

In [3]:
from datasets import Dataset

processed_final_test_dataset = Dataset.load_from_disk(os.path.join(local_DEV_TEST_hf_dataset_dir, "test"))

print(f"\n\033[32m{len(processed_final_test_dataset)}\033[0m\n")

processed_final_test_dataset[4]


1500



{'id': 'ASQE_4',
 'task_type': 'ASQE',
 'entry_id': '4',
 'original_id': '2_4496_234',
 'input': 'Studied for the exam literally the day of... ended with a 90% lol. Not as interesting as I thought it would be tho. The textbook is dry, and the lectures put me to sleep. Overall a good mark booster, but not particularly useful.',
 'prompt': '### Instruction:\nGiven the input text, extract ALL pairs of opinion expressions and their corresponding aspect terms about the course, staff, or university. Then classify the category and sentiment for each aspect-opinion pair.\nOpinion expressions are words/phrases expressing evaluation, feeling, or judgment (including both explicit and implicit opinions, not objective facts).\nAspect terms are opinion targets. Only use a pronoun if you cannot find a direct aspect term in the same sentence or adjacent context. \nEach aspect-opinion-category-sentiment combination is a quadruplet. \n\n**Rules:**\n- Extract EVERY opinion in the text, including both exp

In [4]:
print(processed_final_test_dataset[4]['text'])

### Task type: aspect-sentiment quadruplet extraction (ASQE)
### Instruction:
Given the input text, extract ALL pairs of opinion expressions and their corresponding aspect terms about the course, staff, or university. Then classify the category and sentiment for each aspect-opinion pair.
Opinion expressions are words/phrases expressing evaluation, feeling, or judgment (including both explicit and implicit opinions, not objective facts).
Aspect terms are opinion targets. Only use a pronoun if you cannot find a direct aspect term in the same sentence or adjacent context. 
Each aspect-opinion-category-sentiment combination is a quadruplet. 

**Rules:**
- Extract EVERY opinion in the text, including both explicit and implicit opinion expressions.
- Extract all opinion and aspect terms VERBATIM and as CONSECUTIVE tokens. 
- Use 'null' for implicit aspects. Opinions cannot be null.
- If an aspect is mapped to multiple opinion expressions, or vice versa, extract each 1:1 pair separately. 
- C

In [5]:
print(f"\n\033[36m{processed_final_test_dataset.unique('task_type')}\033[0m\n")
processed_final_test_dataset


['OE', 'AOPE', 'AOC', 'ASTE', 'ASQE']



Dataset({
    features: ['id', 'task_type', 'entry_id', 'original_id', 'input', 'prompt', 'entry_list', 'output', 'text', 'type'],
    num_rows: 1500
})

## <font color='magenta'>Make a small subset for testing</font>

In [6]:
if try_small_sample_with_this_script:

    processed_final_test_dataset = processed_final_test_dataset.select(range(20))
    
    print(f"\033[1;30;45mTrimmed dataset for testing, total entries = {len(processed_final_test_dataset)}\033[0m\n")
else:
    print(f"\033[1;30;42mUse full test set of {len(processed_final_test_dataset)} entries. \033[0m\n")
    
processed_final_test_dataset

Trimmed dataset for testing, total entries = 20



Dataset({
    features: ['id', 'task_type', 'entry_id', 'original_id', 'input', 'prompt', 'entry_list', 'output', 'text', 'type'],
    num_rows: 20
})


# <font color='gold'>Evaluate merged model</font>

In [7]:
###############################################
# Load pre-trained model & set up trainer
###############################################

tokenizer.padding_side = 'left'  # left for generation, right for training

merged_model = AutoModelForCausalLM.from_pretrained(model_checkpoint_path, **evaluation_model_kwargs)

print(f"\n\033[33meval model kwargs:\033[0m\n{evaluation_model_kwargs}\n")
print(f"\033[33mTokenizer padding:\033[0m  {tokenizer.pad_token}, {tokenizer.padding_side}")

# if config specifies, move model to GPU 
# if evaluation_model_to_gpu: 
merged_model = merged_model.to(device) 
print(f"\033[35m\n{'*'*20}\nmoving merged_model to {device}\n{'*'*20}\n\033[0m")
    
print_gpu_utilization()
    
print(f"Evaluating pre-trained model on test dataset")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


eval model kwargs:
{'use_cache': False, 'trust_remote_code': False, 'attn_implementation': 'flash_attention_2', 'torch_dtype': torch.bfloat16, 'device_map': {'': 0}}

Tokenizer padding:  <|endoftext|>, left

********************
moving merged_model to cuda
********************

GPU memory occupied: 8662 MB

Evaluating pre-trained model on test dataset


In [8]:
generation_args

{'max_new_tokens': 1024,
 'return_full_text': False,
 'temperature': None,
 'top_p': None,
 'top_k': None,
 'do_sample': False}

In [9]:
###############################################
# Evaluate fine-tuned model on ABSA tasks
###############################################

inference_pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, torch_dtype=evaluation_model_kwargs['torch_dtype']) 

model_output = evaluator.evaluate_model(pipeline=inference_pipeline,
                                        generation_args=generation_args,
                                        test_dataset=processed_final_test_dataset,
                                        view_result=evaluation_setting['eval_setting_view_sample'],
                                        view_size=evaluation_setting['eval_setting_view_size'],
                                        handle_format_error = evaluation_setting['handle_format_error'],
                                        show_table=evaluation_setting['eval_setting_show_table'],
                                        return_input_dict=True,
                                        show_first_5_messages=True
                                        )

Device set to use cuda:0


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

0 (OE):  [{'role': 'system', 'content': 'You are an expert in aspect-based sentiment analysis.'}, {'role': 'user', 'content': '### Task type: opinion extraction (OE)\n### Instruction: \nGiven the input text, extract ALL opinion expressions about the course, staff, or university. \nOpinion expressions are words/phrases expressing evaluation, feeling, or judgment (including both explicit and implicit opinions, not objective facts).\n\n**Rules:**\n- Extract each opinion expression VERBATIM and as CONSECUTIVE tokens.\n- Extract EVERY opinion in the text, including both explicit and implicit opinion expressions.\n- Use these specific tags for each extracted opinion expression: <opn>opinion expressions</opn>\n\n**Critical formatting requirements:**\n- Output MUST be a valid Python list\n- Tag-wrapped units MUST be separated by commas\n\n**Output format:** \n[<opn>...</opn>, <opn>...</opn>, ..., <opn>...</opn>]\n\n### Examples:\nInput: "The professor was knowledgeable but the assignments were

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


9.    ASQE   
Input:
The campus is very green, and a beautiful place to work. There are outstanding facilities in each department and a staff that are clearly striving for the best in every student.
ground_truth:
['<asp>campus</asp><opn>a beautiful place to work</opn><cat>University - Campus & facilities</cat><sen>positive</sen>', '<asp>campus</asp><opn>very green</opn><cat>University - Campus & facilities</cat><sen>positive</sen>', '<asp>facilities</asp><opn>outstanding</opn><cat>University - Campus & facilities</cat><sen>positive</sen>', '<asp>staff</asp><opn>clearly striving for the best in every student</opn><cat>Staff - Overall</cat><sen>positive</sen>']
raw_pred:
{'"[<asp>campus</asp><opn>very green</opn><cat>University - Campus & facilities</cat><sen>positive</sen>, <asp>campus</asp><opn>beautiful place to work</opn><cat>University - Campus & facilities</cat><sen>positive</sen>, <asp>facilities</asp><opn>outstanding</opn><cat>University - Campus & facilities</cat><sen>positive</

In [10]:
eval_results = model_output['results']

print(f"sft_output:             {model_output.keys()}\n")
print(f"sft_output['results']:  {eval_results.keys()}\n")

sft_output:             dict_keys(['all_preds', 'all_labels', 'eval_input', 'results'])

sft_output['results']:  dict_keys(['dataframes', 'match_details_dict', 'input_dict'])



### <font color='magenta'>Write eval_input and processed input_dict to file</font>

In [11]:
eval_input = model_output.get('eval_input', dict())

eval_input_dict = eval_results.get('input_dict', dict())

print(len(eval_input), len(eval_input_dict))


# if not os.path.exists(eval_input_dict_dir):
print(test_input_dict_dir)
with open (test_input_dict_dir, 'a', encoding='utf-8') as dictfile:  # use 'a' mode to append instead of overwrite. 
    print(f"eval_input = {eval_input}", file=dictfile)    
    print(f"sft_input_dict = {eval_input_dict}", file=dictfile)
    print(f"\033[33mwriting to dict: {test_input_dict_dir}\033[0m\n")
# else:
#     print(f"\n\u001b[4m\u001b[44m{eval_input_dict_dir} file already written, check if need overwrite\033[0m\n")


20 20
../OUTPUTS/OUTPUT_1_final_test_output_files/_demo/test_input_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).py
writing to dict: ../OUTPUTS/OUTPUT_1_final_test_output_files/_demo/test_input_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).py



In [12]:
eval_result_dfdict = eval_results.get('dataframes', dict())

eval_result_dfdict['task_summary_df'] = evaluator._transpose_df(eval_result_dfdict['task_summary_df'], 'merged')

print(f"{eval_result_dfdict.keys()}\n\n")

dict_keys(['task_summary_df', 'entries_metrics_df', 'pairs_metrics_df', 'match_details_df'])




### <font color='magenta'>Write df to dict</font>

In [13]:
for df_name, df in eval_result_dfdict.items(): 
    sheetname_df_dict[f"SFT_{df_name}"] = df
    print(f'\033[36m{df_name:<20}   len = \033[0m{len(df)}')

task_summary_df        len = 60
entries_metrics_df     len = 20
pairs_metrics_df       len = 90
match_details_df       len = 96


In [14]:
eval_result_dfdict['task_summary_df']

,task_type,merged OE,merged AOPE,merged AOC,merged ASTE,merged ASQE
0,entry_count,4,4,4,4,4
1,total_pred,20,18,20,18,20
2,total_gold,18,18,18,18,18
3,total_matched,18,18,18,18,18
4,unit_match_TP,15,15,10,15,11
5,unit_match_FP,5,3,10,3,9
6,unit_match_FN,3,3,8,3,7
7,unit_match_micro_precision,0.75,0.833333,0.5,0.833333,0.55
8,unit_match_micro_recall,0.833333,0.833333,0.555556,0.833333,0.611111
9,unit_match_micro_f1,0.789474,0.833333,0.526316,0.833333,0.578947


In [15]:
eval_result_dfdict['pairs_metrics_df']

,entry_id,task_type,pair_id,is_unit_match,aspect_score,opinion_score,category_score,sentiment_score,gold_item,pred_item,weighted_score
0,0,OE,0-g3-p0,True,NaN,1.000000,NaN,NaN,<opn>good mark booster</opn>,<opn>good mark booster</opn>,1.000000
1,0,OE,0-g4-p1,True,NaN,1.000000,NaN,NaN,<opn>not particularly useful</opn>,<opn>not particularly useful</opn>,1.000000
2,0,OE,0-g0-p2,True,NaN,0.933333,NaN,NaN,<opn>Not as interesting as I thought it would ...,<opn>Not as interesting as I thought it would ...,0.933333
3,0,OE,0-g1-p3,True,NaN,1.000000,NaN,NaN,<opn>dry</opn>,<opn>dry</opn>,1.000000
4,0,OE,0-g2-p4,True,NaN,1.000000,NaN,NaN,<opn>put me to sleep</opn>,<opn>put me to sleep</opn>,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
85,17,AOC,17-g1-p1,False,1.0,0.000000,1.0,NaN,<asp>null</asp><opn>forgetting everything</opn...,<asp>null</asp><opn>forget everything</opn><ca...,0.666667
86,18,ASTE,18-g0-p0,True,1.0,0.666667,NaN,1.0,<asp>null</asp><opn>can get a decent grade</op...,<asp>null</asp><opn>decent grade</opn><sen>pos...,0.888889
87,18,ASTE,18-g1-p1,False,1.0,0.000000,NaN,1.0,<asp>null</asp><opn>forgetting everything</opn...,<asp>null</asp><opn>forget everything</opn><se...,0.666667
88,19,ASQE,19-g0-p0,False,1.0,0.666667,0.3,1.0,<asp>null</asp><opn>can get a decent grade</op...,<asp>null</asp><opn>decent grade</opn><cat>Cou...,0.741667


In [16]:
eval_result_dfdict['match_details_df']

,entry_id,Task,PairID,is_optimal_match,is_unit_match,Input Text,Match Quality,Gold-Aspect,Pred-Aspect,Aspect Score,Gold-Opinion,Pred-Opinion,Opinion Score,Gold-Category,Pred-Category,Category Score,Gold-Sentiment,Pred-Sentiment,Sentiment Score,Overall Score
0,0,OE,0-g0-p2,True,True,Studied for the exam literally the day of... e...,Strong Match,NaN,NaN,NaN,Not as interesting as I thought it would be tho,Not as interesting as I thought it would be,0.933333,NaN,NaN,NaN,NaN,NaN,NaN,0.933333
1,0,OE,0-g1-p3,True,True,Studied for the exam literally the day of... e...,Full Match,NaN,NaN,NaN,dry,dry,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2,0,OE,0-g2-p4,True,True,Studied for the exam literally the day of... e...,Full Match,NaN,NaN,NaN,put me to sleep,put me to sleep,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
3,0,OE,0-g3-p0,True,True,Studied for the exam literally the day of... e...,Full Match,NaN,NaN,NaN,good mark booster,good mark booster,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
4,0,OE,0-g4-p1,True,True,Studied for the exam literally the day of... e...,Full Match,NaN,NaN,NaN,not particularly useful,not particularly useful,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,17,AOC,17-g1-p1,True,False,"You can get a decent grade, but end up forgett...",Good Match,null,null,1.0,forgetting everything,forget everything,0.000000,Course - Overall,Course - Overall,1.0,NaN,NaN,NaN,0.666667
92,18,ASTE,18-g0-p0,True,True,"You can get a decent grade, but end up forgett...",Strong Match,null,null,1.0,can get a decent grade,decent grade,0.666667,NaN,NaN,NaN,positive,positive,1.0,0.888889
93,18,ASTE,18-g1-p1,True,False,"You can get a decent grade, but end up forgett...",Good Match,null,null,1.0,forgetting everything,forget everything,0.000000,NaN,NaN,NaN,negative,negative,1.0,0.666667
94,19,ASQE,19-g0-p0,True,False,"You can get a decent grade, but end up forgett...",Good Match,null,null,1.0,can get a decent grade,decent grade,0.666667,Course - Difficulty,Course - Overall,0.3,positive,positive,1.0,0.741667


In [17]:
eval_input_dict

{0: {'task_type': 'OE',
  'input_text': 'Studied for the exam literally the day of... ended with a 90% lol. Not as interesting as I thought it would be tho. The textbook is dry, and the lectures put me to sleep. Overall a good mark booster, but not particularly useful.',
  'label': ['<opn>Not as interesting as I thought it would be tho</opn>',
   '<opn>dry</opn>',
   '<opn>put me to sleep</opn>',
   '<opn>good mark booster</opn>',
   '<opn>not particularly useful</opn>'],
  'pred': ['<opn>good mark booster</opn>',
   '<opn>not particularly useful</opn>',
   '<opn>Not as interesting as I thought it would be</opn>',
   '<opn>dry</opn>',
   '<opn>put me to sleep</opn>',
   '<opn>good mark booster</opn>',
   '<opn>not particularly useful</opn>']},
 1: {'task_type': 'AOPE',
  'input_text': 'Studied for the exam literally the day of... ended with a 90% lol. Not as interesting as I thought it would be tho. The textbook is dry, and the lectures put me to sleep. Overall a good mark booster, but

# <font color='gold'>Write to Excel</font>

In [18]:
df_dictToExcel(excelfilename=test_xlsx_file_dir, sheetname_df_dict=sheetname_df_dict, freezeheader=True, headerfilter=True, locksheet=False, editable_range=None)
print(f"\n{green_bg}Done!  :)\033[30m") 

SFT_task_summary_df written to ../OUTPUTS/OUTPUT_2a_final_test_output_FTS-0BP_evaluation/_demo/test_results_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).xlsx.
SFT_entries_metrics_df written to ../OUTPUTS/OUTPUT_2a_final_test_output_FTS-0BP_evaluation/_demo/test_results_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).xlsx.
SFT_pairs_metrics_df written to ../OUTPUTS/OUTPUT_2a_final_test_output_FTS-0BP_evaluation/_demo/test_results_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).xlsx.
SFT_match_details_df written to ../OUTPUTS/OUTPUT_2a_final_test_output_FTS-0BP_evaluation/_demo/test_results_merged_lora_phi4mini_A46.2_train2000_R64_multitask_OE-AOPE-AOC-ASTE-ASQE_fewshot_(2025-11-06).xlsx.



Done!  :)
